Create a model that will predict whether a person does or does not have diabetes. Use the diabetes.csv dataset. The target column in the dataset is "Outcome". Assume no features leak information about the target.

Your solution should include the below. You may use whichever python libraries you wish to complete the task:
1. Feature engineering
2. Model fitting and performance evaluation
3. A function that takes as arguments: a model, train data, test data, and returns the model's predictions on the test data
4. A function that takes a set of predictions and true values and that validates the predictions using appropriate metrics
5. Anything else you feel is necessary for modelling or improving the performance of your model

**Note:** The data is a known dataset but with added noise.

### 1. Data Exploration

In [1]:
import pandas as pd
import numpy as np

# load dataset and look at top datapoints
df = pd.read_csv("../data/test_diabetes.csv", sep=";")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,148.0,72.0,35.0,0,33.6,0.627,50.0,1
1,1.0,85.0,66.0,29.0,0,26.6,0.351,31.0,0
2,8.0,183.0,64.0,0.0,0,23.3,0.672,32.0,1
3,1.0,89.0,66.0,23.0,94,28.1,0.167,21.0,0
4,0.0,NaN,40.0,35.0,168,43.1,2.288,NaN,1


In [2]:
len(df)

768

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               731 non-null    float64
 1   Glucose                   730 non-null    float64
 2   BloodPressure             734 non-null    float64
 3   SkinThickness             734 non-null    float64
 4   Insulin                   717 non-null    object 
 5   BMI                       733 non-null    float64
 6   DiabetesPedigreeFunction  728 non-null    float64
 7   Age                       717 non-null    float64
 8   Outcome                   768 non-null    object 
dtypes: float64(7), object(2)
memory usage: 54.1+ KB


In [4]:
# look for missing values AKA NaN
df.isna().sum()

Pregnancies                 37
Glucose                     38
BloodPressure               34
SkinThickness               34
Insulin                     51
BMI                         35
DiabetesPedigreeFunction    40
Age                         51
Outcome                      0
dtype: int64

In [5]:
# Replace NaN values with last valid observation
df = df.fillna(method='ffill')

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,148.0,72.0,35.0,0,33.6,0.627,50.0,1
1,1.0,85.0,66.0,29.0,0,26.6,0.351,31.0,0
2,8.0,183.0,64.0,0.0,0,23.3,0.672,32.0,1
3,1.0,89.0,66.0,23.0,94,28.1,0.167,21.0,0
4,0.0,89.0,40.0,35.0,168,43.1,2.288,21.0,1


In [7]:
df.isna().sum()

Pregnancies                 1
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

The first pregnancies record still shows a NaN value because there is no "Last Observation" for the first record.
We can correct this filling using the "Next Observation"

In [8]:
df = df.fillna(method='bfill')
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [9]:
# Find categorical variables
object_cols = [col for col in df.columns if df[col].dtype == "object"]
object_cols

['Insulin', 'Outcome']

In [10]:
# The feature to predict is Outcome so we will make a y dataframe with it
y = df["Outcome"]
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: object

In [11]:
# we need to replace Nss and Ys with 1s and 0s
y.unique()

array(['1', '0', 'N', 'Y'], dtype=object)

In [12]:
y = y.replace(["Y"],1)
y = y.replace(["N"],0)
y.unique()

array(['1', '0', 0, 1], dtype=object)

In [13]:
# Now we need to replace string 1s and string 0s into 1s and 0s
y = y.replace(["1"],1)
y = y.replace(["0"],0)
y.unique()

array([1, 0], dtype=int64)

In [14]:
# We will drop "Outcome" from the preprocessing dataframe
df = df.drop(columns=["Outcome"])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1.0,148.0,72.0,35.0,0,33.6,0.627,50.0
1,1.0,85.0,66.0,29.0,0,26.6,0.351,31.0
2,8.0,183.0,64.0,0.0,0,23.3,0.672,32.0
3,1.0,89.0,66.0,23.0,94,28.1,0.167,21.0
4,0.0,89.0,40.0,35.0,168,43.1,2.288,21.0


Now there are no missing values, but there is an object type column we should replace with a numeric type

In [15]:
df = df.apply(pd.to_numeric, errors="coerce")

In [16]:
df["Insulin"]

0        0.0
1        0.0
2        0.0
3       94.0
4      168.0
       ...  
763    180.0
764      NaN
765    112.0
766      NaN
767      NaN
Name: Insulin, Length: 768, dtype: float64

Now we have NaN values again, and we will fill them as we did the last time.

In [17]:
df = df.fillna(method='ffill')
# To make sure we don't miss any
df = df.fillna(method='bfill')

In [18]:
# Now we don't have any NaN values
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

 ### 2. Feature Engineering

In [19]:
from sklearn.feature_selection import SelectKBest, f_classif

features = df.columns
features

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')

In [20]:
X = df[features]

In [21]:
# Split data into train and valid sets
from sklearn.model_selection import train_test_split 

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=686) 

In [22]:
# Analyze features
selector = SelectKBest(f_classif, k=5)

In [23]:
X_new = selector.fit_transform(X_train, y_train)
X_new

array([[1.20e+01, 1.40e+02, 3.25e+02, 3.92e+01, 5.28e-01],
       [6.00e+00, 1.25e+02, 1.20e+02, 3.00e+01, 4.64e-01],
       [0.00e+00, 1.01e+02, 7.10e+01, 2.46e+01, 7.67e-01],
       ...,
       [3.00e+00, 1.73e+02, 1.85e+02, 3.38e+01, 9.70e-01],
       [1.00e+00, 9.60e+01, 1.52e+02, 2.24e+01, 2.07e-01],
       [1.00e+00, 1.09e+02, 1.82e+02, 2.54e+01, 9.47e-01]])

In [24]:
# Show the features we chose and zero out everything else
selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=X_train.index, 
                                 columns=features)
selected_features.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
375,12.0,140.0,0.0,0.0,325.0,39.2,0.528,0.0
217,6.0,125.0,0.0,0.0,120.0,30.0,0.464,0.0
83,0.0,101.0,0.0,0.0,71.0,24.6,0.767,0.0
145,0.0,154.0,0.0,0.0,63.0,0.0,0.572,0.0
596,0.0,67.0,0.0,0.0,185.0,45.3,0.682,0.0


In [25]:
# Dropped columns have values of all 0s, so var is 0, drop them
selected_columns = selected_features.columns[selected_features.var() != 0]
selected_columns

Index(['Pregnancies', 'Glucose', 'Insulin', 'BMI', 'DiabetesPedigreeFunction'], dtype='object')

Make new training and validation dataframes with the selected columns

In [26]:
fit_X_train = X_train[selected_columns]
fit_X_train

,Pregnancies,Glucose,Insulin,BMI,DiabetesPedigreeFunction
375,12.0,140.0,325.0,39.2,0.528
217,6.0,125.0,120.0,30.0,0.464
83,0.0,101.0,71.0,24.6,0.767
145,0.0,154.0,63.0,0.0,0.572
596,0.0,67.0,185.0,45.3,0.682
...,...,...,...,...,...
134,2.0,96.0,49.0,21.1,0.647
747,1.0,81.0,57.0,46.3,1.096
716,3.0,173.0,185.0,33.8,0.970
106,1.0,96.0,152.0,22.4,0.207


In [27]:
fit_X_valid = X_valid[selected_columns]
fit_X_valid

,Pregnancies,Glucose,Insulin,BMI,DiabetesPedigreeFunction
24,9.0,143.0,146.0,36.6,0.254
69,1.0,146.0,38.0,28.9,0.189
354,3.0,90.0,43.0,42.7,0.559
267,2.0,128.0,67.0,40.0,1.101
558,11.0,110.0,215.0,46.2,0.126
...,...,...,...,...,...
590,11.0,111.0,156.0,46.8,0.925
198,4.0,109.0,99.0,34.8,0.905
363,4.0,146.0,325.0,38.5,0.520
520,2.0,68.0,66.0,25.0,0.187


### 3. Model fitting

In [28]:
# Using the XGBoost Model
from xgboost import XGBClassifier

# Define the model
# n_jobs is the computer's cores 
model = XGBClassifier(n_estimators=1000, learning_rate=0.025, n_jobs=8, random_state=686)

In [29]:
def model_predict(model, X_train, X_valid, y_train, y_valid):
    '''
    Returns model predictions
    '''
    # Fit the model
    model.fit(X_train, y_train, 
              early_stopping_rounds=10,
              eval_set=[(X_valid, y_valid)],
              verbose=False)
    
    predictions = model.predict(X_valid)
    
    return predictions

In [30]:
# Compute model predictions using the custom function
preds = model_predict(model, fit_X_train, fit_X_valid, y_train, y_valid)
preds

array([1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
      dtype=int64)

### 4. Performance evaluation

In [31]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

In [32]:
def validate(real_values, predictions):
    '''
    A function that takes a set of predictions and true values and that validates the predictions using appropriate metrics
    '''
    mae_score = mae(real_values, predictions)
    mse_score = mse(real_values, predictions)
    print("Mean Absolute Error:", mae_score)
    print("Mean Squared Error:", mse_score)    
    

In [33]:
validate(y_valid, preds)

Mean Absolute Error: 0.2922077922077922
Mean Squared Error: 0.2922077922077922


This results suggests about **30% model accuracy**

In [34]:
# Make a dataframe just to see how well we did
results_df = pd.DataFrame({"Actual": y_valid, "Predicted": preds})
results_df

,Actual,Predicted
24,1,1
69,0,1
354,0,0
267,0,1
558,0,0
...,...,...
590,1,1
198,1,0
363,1,0
520,0,0


#### Try a different model using all features

In [35]:
preds2 = model_predict(model, X_train, X_valid, y_train, y_valid)
preds2

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0],
      dtype=int64)

In [36]:
validate(y_valid, preds2)

Mean Absolute Error: 0.2727272727272727
Mean Squared Error: 0.2727272727272727


This suggests a model using all features is slightly worse than the original, with about **27% accuracy**

### 5. Hyper-Parameter Tuning

#### Trying with all features, less estimators

In [37]:
model_rev = XGBClassifier(n_estimators=600, learning_rate=0.01, n_jobs=8, random_state=686)

preds3 = model_predict(model_rev, X_train, X_valid, y_train, y_valid)

validate(y_valid, preds3)

Mean Absolute Error: 0.2922077922077922
Mean Squared Error: 0.2922077922077922


This suggests a model using all features but slightly different hyperparameters is actually as accurate as the model with the parametrically selected features , with about **30% accuracy**

#### Trying with *selected* features, less estimators

In [38]:
model_rev2 = XGBClassifier(n_estimators=600, learning_rate=0.01, n_jobs=8, random_state=686)

preds4 = model_predict(model_rev2, fit_X_train, fit_X_valid, y_train, y_valid)

validate(y_valid, preds4)

Mean Absolute Error: 0.2792207792207792
Mean Squared Error: 0.2792207792207792


This suggests slightly worse results, using the selected features.

#### Trying with *selected* features, smaller learning rate, less estimators

In [39]:
model_rev3 = XGBClassifier(n_estimators=600, learning_rate=0.002, n_jobs=8, random_state=686, verbosity=0, eval_metric="error")

preds5 = model_predict(model_rev3, fit_X_train, fit_X_valid, y_train, y_valid)

validate(y_valid, preds5)

Mean Absolute Error: 0.2922077922077922
Mean Squared Error: 0.2922077922077922


This also suggests model accuracy of about **30%** but it also suggests that the model will not significantly improve anymore, given that the global minima appears to have been reached with different hyperparameters. This can also indicate that the data is leaking or that more feature engineering is required. Also, grid search could be done, given the time, to find if better hyperparamers can be used to increase accuracy a few more percentage points.